Goal: Determine quality of a car

let's analyze the dataset first
- read in as pandas
- histogram each of the columns

Now for the learning
- then do the preprocessing
- classification with random forest
- classification with Naive Bayes

Run metrics
- classification reports
- can I graph some confusion matricies?